In [1]:
### IMPORTS ###
import os
import xml.etree.ElementTree as ET
from PIL import Image
import shutil
import random

In [2]:
annotation_folder = "car-license-plate-detection/annotations"
output_folder = "car-license-plate-detection/yolo_annotations" 
os.makedirs(output_folder, exist_ok=True)

# Only one class in this dataset
class_map = {'licence':0}

In [ ]:
# Annotations are in PASCAL VOC format.
# Need to convert to YOLO format

def convert_annotation(xml_file, output_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Image dimensions from <size> tag
    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    with open(output_file, 'w') as out_file:
        for obj in root.iter('object'):
            label = obj.find('name').text.lower()
            
            # This should be unecessary, but safety first :)
            if label not in class_map:
                continue
            
            # YOLO needs class id
            class_id = class_map[label]

            # Get bbox coords
            xmlbox = obj.find('bndbox')
            xmin = float(xmlbox.find('xmin').text)
            ymin = float(xmlbox.find('ymin').text)
            xmax = float(xmlbox.find('xmax').text)
            ymax = float(xmlbox.find('ymax').text)

            # YOLO requires normalized values (x,y,w,h)
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            w = (xmax - xmin) / width
            h = (ymax - ymin) / height

            # Write new file in YOLO format
            out_file.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

for xml_file in os.listdir(annotation_folder):
    if xml_file.endswith('.xml'):
        xml_path = os.path.join(annotation_folder, xml_file)
        #Cars0.xml -> Cars0.txt
        output_file = os.path.join(output_folder, os.path.splitext(xml_file)[0] + '.txt')
        convert_annotation(xml_path, output_file)

print("Finished converting. Output directory:", output_folder)

Finished converting. Output directory: output


In [ ]:
# Need to split dataset

# Base directories 
base_images_dir = 'car-license-plate-detection/images'
base_xml_dir = 'car-license-plate-detection/annotations'         
base_yolo_dir = 'car-license-plate-detection/yolo_annotations'     

# New Directories 
train_images_dir = 'car-license-plate-detection/train/images'
val_images_dir   = 'car-license-plate-detection/val/images'
test_images_dir  = 'car-license-plate-detection/test/images'

# Ultralytics expects /labels directory
train_yolo_dir   = 'car-license-plate-detection/train/labels'
val_yolo_dir     = 'car-license-plate-detection/val/labels'
test_yolo_dir    = 'car-license-plate-detection/test/labels'

train_xml_dir    = 'car-license-plate-detection/train/xml_annotations'
val_xml_dir      = 'car-license-plate-detection/val/xml_annotations'  
test_xml_dir     = 'car-license-plate-detection/test/xml_annotations' 

# Create destination directories if they don't exist
dest_dirs = [train_images_dir, val_images_dir, test_images_dir,
             train_yolo_dir, val_yolo_dir, test_yolo_dir,
             train_xml_dir, val_xml_dir, test_xml_dir]
for d in dest_dirs:
    os.makedirs(d, exist_ok=True)

# List all image files (e.g., png, jpg, jpeg)
image_files = [f for f in os.listdir(base_images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
random.shuffle(image_files)

# Define split ratios (70% train, 15% validation, 15% test)
n_total = len(image_files)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)
n_test  = n_total - n_train - n_val 

train_files = image_files[:n_train]
val_files   = image_files[n_train:n_train+n_val]
test_files  = image_files[n_train+n_val:]

def copy_files(file_list, src_img_dir, src_yolo_dir, src_xml_dir, dst_img_dir, dst_yolo_dir, dst_xml_dir):
    for file in file_list:
        # Copy image file
        shutil.copy(os.path.join(src_img_dir, file), os.path.join(dst_img_dir, file))
        
        # YOLO ANNOTATIONS
        yolo_file = os.path.splitext(file)[0] + '.txt'
        src_yolo_file = os.path.join(src_yolo_dir, yolo_file)
        if os.path.exists(src_yolo_file):
            shutil.copy(src_yolo_file, os.path.join(dst_yolo_dir, yolo_file))
        
        # XML ANNOTATIONS
        xml_file = os.path.splitext(file)[0] + '.xml'
        src_xml_file = os.path.join(src_xml_dir, xml_file)
        if os.path.exists(src_xml_file):
            shutil.copy(src_xml_file, os.path.join(dst_xml_dir, xml_file))

# Training files
copy_files(train_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           train_images_dir, train_yolo_dir, train_xml_dir)

# Validation files
copy_files(val_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           val_images_dir, val_yolo_dir, val_xml_dir)

# Test Files
copy_files(test_files, base_images_dir, base_yolo_dir, base_xml_dir, 
           test_images_dir, test_yolo_dir, test_xml_dir)

In [3]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data='data.yaml',
    epochs=20,
    batch=16,
    imgsz=640 # Yolo assumes 640
)

Ultralytics 8.3.94  Python-3.9.7 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning C:\Users\aidan\OneDrive\Documents\uofc\enel645\ENEL645_Project_Group2\segment\car-license-plate-detection\train\labels.cache... 303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 303/303 [00:00<?, ?it/s]
val: Scanning C:\Users\aidan\OneDrive\Documents\uofc\enel645\ENEL645_Project_Group2\segment\car-license-plate-detection\val\labels.cache... 64 images, 0 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train7
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      2.52G      1.499      3.256      1.306         32        640: 100%|██████████| 19/19 [00:03<00:00,  5.14it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.21it/s]

                   all         64         70    0.00318      0.871      0.253      0.134


...

      20/20      2.62G      1.128     0.9182      1.091         15        640: 100%|██████████| 19/19 [00:01<00:00, 10.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.80it/s]

                   all         64         70      0.923      0.855      0.887      0.522



20 epochs completed in 0.021 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 6.2MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.94  Python-3.9.7 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3070 Ti, 8191MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.00it/s]


                   all         64         70      0.923      0.855      0.887      0.521
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs\detect\train7


c:\Users\aidan\OneDrive\Documents\uofc\enel645\ENEL645_Project_Group2\segment
